In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import dask.dataframe as dd
from geopy.distance import geodesic
import string

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [159]:
dir = '../csv/'
fname = 'credit_card_fraud_sample_2019_Dec_100K'
df = pd.read_csv(dir+fname+'.csv')

#df = pd.read_csv('../csv/credit_card_fraud_2019_q1_15K_rows.csv')
#df = pd.read_csv('../csv/credit_card_fraud_2019_Dec.csv')
#df = pd.read_csv('../csv/credit_card_fraud_sample_2019_Dec_100K.csv')



In [160]:
df = df.drop(columns=['quarter']) # quarter added while spliting the larger dataset
df = df.rename(columns = {'Unnamed: 0':'id'})
df.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long
0,18832098,087-12-2118,639017474813,Zachary,Parks,M,94238 Jacob Cliff Apt. 458,Columbia,MO,65203,38.9348,-92.3639,139474,"Designer, exhibition/display",1970-10-23,941725909341,adults_50up_male_urban.json,a36eecc709911793fe140ba7d29404ca,2019-12-26,06:36:03,1577322363,entertainment,3.65,0,"fraud_Turcotte, McKenzie and Koss",37.978633,-91.813002
1,24801519,267-50-6963,6567701540795312,Jeremy,Soto,M,29865 Bush Centers,Goldendale,WA,98620,45.8320,-120.8130,7123,Psychiatric nurse,1983-08-03,856035171395,adults_2550_male_urban.json,5c0acce002c0e0b56de66d9de0cbff80,2019-12-28,22:53:34,1577553814,shopping_pos,5.25,0,fraud_Hermann and Sons,45.829641,-121.389115
2,31141574,411-62-3428,180022197664580,Samuel,Mcdonald,M,4570 Ruth Harbors,Palo Alto,CA,94306,37.4180,-122.1274,92558,Occupational psychologist,1980-04-27,401937847226,adults_2550_male_urban.json,c363df118159e47c9a46ff906471c0c0,2019-12-23,13:23:24,1577087604,shopping_pos,25.31,0,fraud_Conroy Ltd,37.825064,-121.644324
3,25784982,841-48-6189,3584095374557286,Nicholas,Davis,M,83759 Shannon Meadows,Kohler,WI,53044,43.7381,-87.7867,2297,Hospital pharmacist,1998-11-13,497586259866,adults_2550_male_rural.json,e33a7e2168a80e7b3a9c7ba239219259,2019-12-03,09:21:51,1575345111,shopping_pos,95.28,0,fraud_Hudson-Grady,43.686522,-86.947295
4,19821729,603-28-2907,6011226654586127,Mark,Sanchez,M,21975 Pam Garden,Birmingham,MI,48009,42.5444,-83.2133,20107,Medical physicist,2001-07-19,765945154350,young_adults_male_urban.json,bf6116234aaf567b13a629cf6b604b0b,2019-12-27,21:56:44,1577464004,shopping_pos,73.33,0,fraud_Dooley Inc,43.031478,-83.903585


In [162]:
print('Shape: ', df.shape)
print('Columns: ', df.columns)
print('Data types: ', df.info())    



Shape:  (100000, 27)
Columns:  Index(['id', 'ssn', 'cc_num', 'first', 'last', 'gender', 'street', 'city',
       'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'acct_num',
       'profile', 'trans_num', 'trans_date', 'trans_time', 'unix_time',
       'category', 'amt', 'is_fraud', 'merchant', 'merch_lat', 'merch_long'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          100000 non-null  int64  
 1   ssn         100000 non-null  object 
 2   cc_num      100000 non-null  int64  
 3   first       100000 non-null  object 
 4   last        100000 non-null  object 
 5   gender      100000 non-null  object 
 6   street      100000 non-null  object 
 7   city        100000 non-null  object 
 8   state       100000 non-null  object 
 9   zip         100000 non-null  int64  
 10  lat         100000 non-null

In [163]:
df.isna().sum()


id            0
ssn           0
cc_num        0
first         0
last          0
gender        0
street        0
city          0
state         0
zip           0
lat           0
long          0
city_pop      0
job           0
dob           0
acct_num      0
profile       0
trans_num     0
trans_date    0
trans_time    0
unix_time     0
category      0
amt           0
is_fraud      0
merchant      0
merch_lat     0
merch_long    0
dtype: int64

In [164]:
#convert to proper datetime
date_cols = ['dob', 'trans_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce') # will use only dates
df['trans_time'] = pd.to_datetime(df['trans_time'], format='%H:%M:%S', errors='coerce') # will use only time

#Convert numericals
df['zip'] = df['zip'].astype(int)
df['amt'] = df['amt'].astype(float)
df['is_fraud'] = df['is_fraud'].astype(int)
df['unix_time'] = df['unix_time'].astype(int)
#df['quarter'] = df['quarter'].astype(int)

#Clean leading and trailing whitespaces
cat_columns = df.select_dtypes(include = ['object']).columns    
for col in cat_columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.strip()
    else:
        df[col] = df[col].str.strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          100000 non-null  int64         
 1   ssn         100000 non-null  object        
 2   cc_num      100000 non-null  int64         
 3   first       100000 non-null  object        
 4   last        100000 non-null  object        
 5   gender      100000 non-null  object        
 6   street      100000 non-null  object        
 7   city        100000 non-null  object        
 8   state       100000 non-null  object        
 9   zip         100000 non-null  int32         
 10  lat         100000 non-null  float64       
 11  long        100000 non-null  float64       
 12  city_pop    100000 non-null  int64         
 13  job         100000 non-null  object        
 14  dob         100000 non-null  datetime64[ns]
 15  acct_num    100000 non-null  int64         
 16  pro

In [165]:
print(df['gender'].value_counts())
df['gender'] = df['gender'].map({'M': 1, 'F': 0})
print(df['gender'].value_counts())

gender
F    51761
M    48239
Name: count, dtype: int64
gender
0    51761
1    48239
Name: count, dtype: int64


In [166]:
# transaction amount, 
print(df['amt'].describe())
#just in case, clip negative values, may change on different datasets, also check for outliers later
df['amt'] = df['amt'].clip(lower=0)
print(df['amt'].describe())

count    100000.000000
mean         67.459051
std         164.191618
min           1.000000
25%           7.490000
50%          34.500000
75%          79.280000
max       11078.860000
Name: amt, dtype: float64
count    100000.000000
mean         67.459051
std         164.191618
min           1.000000
25%           7.490000
50%          34.500000
75%          79.280000
max       11078.860000
Name: amt, dtype: float64


In [167]:
#trans_num should be overal unique, but check for duplicates, if any consider removing them
duplicates = df['trans_num'].duplicated(keep=False)
print('Duplicate transactions: ', duplicates.sum())

Duplicate transactions:  0


In [168]:
df.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long
0,18832098,087-12-2118,639017474813,Zachary,Parks,1,94238 Jacob Cliff Apt. 458,Columbia,MO,65203,38.9348,-92.3639,139474,"Designer, exhibition/display",1970-10-23,941725909341,adults_50up_male_urban.json,a36eecc709911793fe140ba7d29404ca,2019-12-26,1900-01-01 06:36:03,1577322363,entertainment,3.65,0,"fraud_Turcotte, McKenzie and Koss",37.978633,-91.813002
1,24801519,267-50-6963,6567701540795312,Jeremy,Soto,1,29865 Bush Centers,Goldendale,WA,98620,45.8320,-120.8130,7123,Psychiatric nurse,1983-08-03,856035171395,adults_2550_male_urban.json,5c0acce002c0e0b56de66d9de0cbff80,2019-12-28,1900-01-01 22:53:34,1577553814,shopping_pos,5.25,0,fraud_Hermann and Sons,45.829641,-121.389115
2,31141574,411-62-3428,180022197664580,Samuel,Mcdonald,1,4570 Ruth Harbors,Palo Alto,CA,94306,37.4180,-122.1274,92558,Occupational psychologist,1980-04-27,401937847226,adults_2550_male_urban.json,c363df118159e47c9a46ff906471c0c0,2019-12-23,1900-01-01 13:23:24,1577087604,shopping_pos,25.31,0,fraud_Conroy Ltd,37.825064,-121.644324
3,25784982,841-48-6189,3584095374557286,Nicholas,Davis,1,83759 Shannon Meadows,Kohler,WI,53044,43.7381,-87.7867,2297,Hospital pharmacist,1998-11-13,497586259866,adults_2550_male_rural.json,e33a7e2168a80e7b3a9c7ba239219259,2019-12-03,1900-01-01 09:21:51,1575345111,shopping_pos,95.28,0,fraud_Hudson-Grady,43.686522,-86.947295
4,19821729,603-28-2907,6011226654586127,Mark,Sanchez,1,21975 Pam Garden,Birmingham,MI,48009,42.5444,-83.2133,20107,Medical physicist,2001-07-19,765945154350,young_adults_male_urban.json,bf6116234aaf567b13a629cf6b604b0b,2019-12-27,1900-01-01 21:56:44,1577464004,shopping_pos,73.33,0,fraud_Dooley Inc,43.031478,-83.903585


In [170]:
#clean textual data
df['merchant'] = df['merchant'].str.lower()
df['merchant'] = df['merchant'].str.replace('fraud_', '')


df['job'] = df['job'].str.lower()
df['job'] = df['job'].str.replace(',', '')


In [171]:
#remove only exact duplicate rows
df = df.drop_duplicates()

In [172]:
# Ensure latitude is between -90 and 90
df['lat'] = df['lat'].clip(-90, 90)
df['merch_lat'] = df['merch_lat'].clip(-90, 90)

# Ensure longitude is between -180 and 180
df['long'] = df['long'].clip(-180, 180)
df['merch_long'] = df['merch_long'].clip(-180, 180)


In [174]:
df['state'] = df['state'].str.upper()
print(df['state'].value_counts())

# Define region lists
NORTHEAST = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA']
MIDWEST = ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']
SOUTH = ['AL', 'AR', 'DE', 'FL', 'GA', 'KY', 'LA', 'MD', 'MS', 'NC', 'OK', 'SC', 'TN', 'TX', 'VA', 'WV', 'DC']
WEST = ['AK', 'AZ', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']

# Create the region dictionary
REGION_DICT = {state: 'Northeast' for state in NORTHEAST}
REGION_DICT.update({state: 'Midwest' for state in MIDWEST})
REGION_DICT.update({state: 'South' for state in SOUTH})
REGION_DICT.update({state: 'West' for state in WEST})

# Define the function to map state to region
def get_region(state):
    if not isinstance(state, str):
        return 'Unknown'
    state = state.strip().upper()
    return REGION_DICT.get(state, 'Unknown')


df['region'] = df['state'].apply(get_region)
print(df['region'].value_counts())

state
CA    11658
TX     7745
NY     6471
FL     5828
IL     4589
PA     4352
OH     3584
GA     3280
NC     3187
MI     3156
NJ     2802
VA     2741
AZ     2307
WA     2256
TN     2028
IN     2028
MA     2000
MD     1923
MO     1818
MN     1772
SC     1767
WI     1662
CO     1523
KY     1424
AL     1373
LA     1356
OR     1215
OK     1202
CT     1183
NV     1050
IA     1000
AR      960
UT      923
MS      863
KS      807
NM      741
NE      604
WV      549
HI      499
ME      473
ID      462
NH      448
DE      339
VT      330
SD      321
MT      313
RI      288
AK      236
ND      221
WY      174
DC      169
Name: count, dtype: int64
region
South        36734
West         23357
Midwest      21562
Northeast    18347
Name: count, dtype: int64


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          100000 non-null  int64         
 1   ssn         100000 non-null  object        
 2   cc_num      100000 non-null  int64         
 3   first       100000 non-null  object        
 4   last        100000 non-null  object        
 5   gender      100000 non-null  int64         
 6   street      100000 non-null  object        
 7   city        100000 non-null  object        
 8   state       100000 non-null  object        
 9   zip         100000 non-null  int32         
 10  lat         100000 non-null  float64       
 11  long        100000 non-null  float64       
 12  city_pop    100000 non-null  int64         
 13  job         100000 non-null  object        
 14  dob         100000 non-null  datetime64[ns]
 15  acct_num    100000 non-null  int64         
 16  pro

### Narrowing some categories

In [176]:
def assign_time_segment(hour):
    if 0 <= hour < 6:
        return 'Late_Night_0-6'
    elif 6 <= hour < 12:
        return 'Morning_6-12'
    elif 12 <= hour < 18:
        return 'Afternoon_12-18'
    else:
        return 'Evening_18-24'

def assign_age_group(age):
    if age < 15:
        return 'child'
    elif age < 20:
        return 'teenager'
    elif age < 40:
        return 'young_adult'
    elif age < 60:
        return 'middle_aged'
    elif age < 80:
        return 'senior'
    else:
        return 'elderly'

#df['trans_date'] = pd.to_datetime(df['trans_date'])
#df['trans_time'] = pd.to_datetime(df['trans_time'])

#df['trans_year'] = df['trans_date'].dt.year
#df['trans_month'] = df['trans_date'].dt.month
#df['trans_day'] = df['trans_date'].dt.day

#df['trans_hour'] = df['trans_time'].dt.hour
#df['trans_minute'] =df['trans_time'].dt.minute
#df['trans_second'] =df['trans_time'].dt.second
df['trans_time_segment'] = df['trans_time'].dt.hour.apply(assign_time_segment)

#df['dob'] = pd.to_datetime(df['dob'])
#df['age'] = df['trans_year']  - df['dob'].dt.year

#add an age column at the time of the transaction
age_df = df['trans_date'].dt.year - pd.to_datetime(df['dob']).dt.year

df['age_group'] = age_df.apply(assign_age_group)

print('age distribution', age_df.describe()) #check for outliers...
print(df['age_group'].value_counts())

age distribution count    100000.000000
mean         40.663180
std          17.850472
min          11.000000
25%          27.000000
50%          38.000000
75%          52.000000
max          91.000000
dtype: float64
age_group
young_adult    43480
middle_aged    30712
senior         12303
teenager        8648
elderly         3703
child           1154
Name: count, dtype: int64


In [177]:
df['category'].value_counts()

category
shopping_pos      79227
entertainment     13845
shopping_net       6742
grocery_pos          74
misc_net             28
gas_transport        24
food_dining          12
misc_pos             12
grocery_net           8
travel                7
home                  7
kids_pets             6
personal_care         6
health_fitness        2
Name: count, dtype: int64

In [178]:
#narrow down the CC categories

def get_cc_type(cc_num):
    # Handle missing values (e.g., None or NaN)
    if pd.isna(cc_num):
        return 'Unknown'
    
    # Convert input to string (handles integers, floats, etc.)
    cc_num = str(cc_num)
    
    # Remove non-digit characters (e.g., spaces, hyphens)
    cc_num = ''.join(filter(str.isdigit, cc_num))
    
    # Return 'Unknown' if the result is empty
    if not cc_num:
        return 'Unknown'
    
    # Check UnionPay: starts with 62
    if len(cc_num) >= 2 and cc_num[:2] == '62':
        return 'UnionPay'
    
    # Check Maestro: starts with 50, 56-69
    elif len(cc_num) >= 2 and (cc_num[:2] == '50' or ('56' <= cc_num[:2] <= '69')):
        return 'Maestro'
    
    # Check RuPay: starts with 60, 6521, 6522
    elif len(cc_num) >= 2 and cc_num[:2] == '60':
        return 'RuPay'
    elif len(cc_num) >= 4 and cc_num[:4] in ['6521', '6522']:
        return 'RuPay'
    
    # Check Laser: starts with 6304, 6706, 6709, 6771
    elif len(cc_num) >= 4 and cc_num[:4] in ['6304', '6706', '6709', '6771']:
        return 'Laser'
    
    # Check Solo: starts with 6334, 6767
    elif len(cc_num) >= 4 and cc_num[:4] in ['6334', '6767']:
        return 'Solo'
    
    # Check Switch: starts with 4903, 4905, 4911, 4936, 564182, 633110, 6333, 6759
    elif len(cc_num) >= 4 and cc_num[:4] in ['4903', '4905', '4911', '4936', '6333', '6759']:
        return 'Switch'
    elif len(cc_num) >= 6 and cc_num[:6] in ['564182', '633110']:
        return 'Switch'
    
    # Check JCB: starts with 3528 to 3589
    elif len(cc_num) >= 4 and '3528' <= cc_num[:4] <= '3589':
        return 'JCB'
    
    # Check Diners Club: starts with 300-305, 36, 38, 54, or 55
    elif len(cc_num) >= 3 and ('300' <= cc_num[:3] <= '305'):
        return 'Diners_Club'
    elif len(cc_num) >= 2 and cc_num[:2] in ['36', '38', '54', '55']:
        return 'Diners_Club'
    
    # Check Visa: starts with 4
    elif len(cc_num) >= 1 and cc_num[0] == '4':
        return 'Visa'
    
    # Check MasterCard: starts with 51-55 or 2221-2720
    elif len(cc_num) >= 2 and ('51' <= cc_num[:2] <= '55'):
        return 'MasterCard'
    elif len(cc_num) >= 4 and ('2221' <= cc_num[:4] <= '2720'):
        return 'MasterCard'
    
    # Check American Express: starts with 34 or 37
    elif len(cc_num) >= 2 and cc_num[:2] in ['34', '37']:
        return 'American_Express'
    
    # Check Discover: starts with 6011, 644-649, or 65
    elif len(cc_num) >= 4 and cc_num[:4] == '6011':
        return 'Discover'
    elif len(cc_num) >= 3 and ('644' <= cc_num[:3] <= '649'):
        return 'Discover'
    elif len(cc_num) >= 2 and cc_num[:2] == '65':
        return 'Discover'
    
    # If no pattern matches, return 'Unknown'
    return 'Unknown'

df['cc_type'] = df['cc_num'].apply(get_cc_type)
print(df['cc_type'].value_counts()) 

cc_type
Visa                33136
Maestro             16495
Unknown             14467
JCB                  9996
Diners_Club          9397
MasterCard           8259
American_Express     8098
Switch                152
Name: count, dtype: int64


In [179]:
# narrow down the profile category to area urban|rural
def get_profile_category(profile):
    if 'urban' in profile:
        return 'urban'  
    elif 'rural' in profile:
        return 'rural'  
    else:       
        return 'unknown'
    

df['profile'] = df['profile'].str.lower()
print(df['profile'].value_counts())
df['area_cat'] = df['profile'].apply(get_profile_category)
print(df['area_cat'].value_counts())



profile
adults_2550_female_urban.json     28388
adults_2550_male_urban.json       24993
adults_50up_female_urban.json     15499
adults_50up_male_urban.json       14835
young_adults_male_urban.json       5984
young_adults_female_urban.json     5119
adults_2550_female_rural.json      1370
adults_50up_female_rural.json      1163
adults_50up_male_rural.json        1134
adults_2550_male_rural.json        1032
young_adults_male_rural.json        261
young_adults_female_rural.json      222
Name: count, dtype: int64
area_cat
urban    94818
rural     5182
Name: count, dtype: int64


In [180]:
df.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long,region,trans_time_segment,age_group,cc_type,area_cat
0,18832098,087-12-2118,639017474813,Zachary,Parks,1,94238 Jacob Cliff Apt. 458,Columbia,MO,65203,38.9348,-92.3639,139474,designer exhibition/display,1970-10-23,941725909341,adults_50up_male_urban.json,a36eecc709911793fe140ba7d29404ca,2019-12-26,1900-01-01 06:36:03,1577322363,entertainment,3.65,0,"turcotte, mckenzie and koss",37.978633,-91.813002,Midwest,Morning_6-12,middle_aged,Maestro,urban
1,24801519,267-50-6963,6567701540795312,Jeremy,Soto,1,29865 Bush Centers,Goldendale,WA,98620,45.8320,-120.8130,7123,psychiatric nurse,1983-08-03,856035171395,adults_2550_male_urban.json,5c0acce002c0e0b56de66d9de0cbff80,2019-12-28,1900-01-01 22:53:34,1577553814,shopping_pos,5.25,0,hermann and sons,45.829641,-121.389115,West,Evening_18-24,young_adult,Maestro,urban
2,31141574,411-62-3428,180022197664580,Samuel,Mcdonald,1,4570 Ruth Harbors,Palo Alto,CA,94306,37.4180,-122.1274,92558,occupational psychologist,1980-04-27,401937847226,adults_2550_male_urban.json,c363df118159e47c9a46ff906471c0c0,2019-12-23,1900-01-01 13:23:24,1577087604,shopping_pos,25.31,0,conroy ltd,37.825064,-121.644324,West,Afternoon_12-18,young_adult,Unknown,urban
3,25784982,841-48-6189,3584095374557286,Nicholas,Davis,1,83759 Shannon Meadows,Kohler,WI,53044,43.7381,-87.7867,2297,hospital pharmacist,1998-11-13,497586259866,adults_2550_male_rural.json,e33a7e2168a80e7b3a9c7ba239219259,2019-12-03,1900-01-01 09:21:51,1575345111,shopping_pos,95.28,0,hudson-grady,43.686522,-86.947295,Midwest,Morning_6-12,young_adult,JCB,rural
4,19821729,603-28-2907,6011226654586127,Mark,Sanchez,1,21975 Pam Garden,Birmingham,MI,48009,42.5444,-83.2133,20107,medical physicist,2001-07-19,765945154350,young_adults_male_urban.json,bf6116234aaf567b13a629cf6b604b0b,2019-12-27,1900-01-01 21:56:44,1577464004,shopping_pos,73.33,0,dooley inc,43.031478,-83.903585,Midwest,Evening_18-24,teenager,Maestro,urban


#Save cleaned df to pickle

In [181]:
df.to_pickle(fname + '_prepared.pkl')

#df = pd.read_pickle(fname + '_prepared.pkl')


# **EDA**

In [206]:
df_eda = pd.read_pickle(fname + '_prepared.pkl')
df_eda.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long,region,trans_time_segment,age_group,cc_type,area_cat
0,18832098,087-12-2118,639017474813,Zachary,Parks,1,94238 Jacob Cliff Apt. 458,Columbia,MO,65203,38.9348,-92.3639,139474,designer exhibition/display,1970-10-23,941725909341,adults_50up_male_urban.json,a36eecc709911793fe140ba7d29404ca,2019-12-26,1900-01-01 06:36:03,1577322363,entertainment,3.65,0,"turcotte, mckenzie and koss",37.978633,-91.813002,Midwest,Morning_6-12,middle_aged,Maestro,urban
1,24801519,267-50-6963,6567701540795312,Jeremy,Soto,1,29865 Bush Centers,Goldendale,WA,98620,45.8320,-120.8130,7123,psychiatric nurse,1983-08-03,856035171395,adults_2550_male_urban.json,5c0acce002c0e0b56de66d9de0cbff80,2019-12-28,1900-01-01 22:53:34,1577553814,shopping_pos,5.25,0,hermann and sons,45.829641,-121.389115,West,Evening_18-24,young_adult,Maestro,urban
2,31141574,411-62-3428,180022197664580,Samuel,Mcdonald,1,4570 Ruth Harbors,Palo Alto,CA,94306,37.4180,-122.1274,92558,occupational psychologist,1980-04-27,401937847226,adults_2550_male_urban.json,c363df118159e47c9a46ff906471c0c0,2019-12-23,1900-01-01 13:23:24,1577087604,shopping_pos,25.31,0,conroy ltd,37.825064,-121.644324,West,Afternoon_12-18,young_adult,Unknown,urban
3,25784982,841-48-6189,3584095374557286,Nicholas,Davis,1,83759 Shannon Meadows,Kohler,WI,53044,43.7381,-87.7867,2297,hospital pharmacist,1998-11-13,497586259866,adults_2550_male_rural.json,e33a7e2168a80e7b3a9c7ba239219259,2019-12-03,1900-01-01 09:21:51,1575345111,shopping_pos,95.28,0,hudson-grady,43.686522,-86.947295,Midwest,Morning_6-12,young_adult,JCB,rural
4,19821729,603-28-2907,6011226654586127,Mark,Sanchez,1,21975 Pam Garden,Birmingham,MI,48009,42.5444,-83.2133,20107,medical physicist,2001-07-19,765945154350,young_adults_male_urban.json,bf6116234aaf567b13a629cf6b604b0b,2019-12-27,1900-01-01 21:56:44,1577464004,shopping_pos,73.33,0,dooley inc,43.031478,-83.903585,Midwest,Evening_18-24,teenager,Maestro,urban


In [207]:
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  100000 non-null  int64         
 1   ssn                 100000 non-null  object        
 2   cc_num              100000 non-null  int64         
 3   first               100000 non-null  object        
 4   last                100000 non-null  object        
 5   gender              100000 non-null  int64         
 6   street              100000 non-null  object        
 7   city                100000 non-null  object        
 8   state               100000 non-null  object        
 9   zip                 100000 non-null  int32         
 10  lat                 100000 non-null  float64       
 11  long                100000 non-null  float64       
 12  city_pop            100000 non-null  int64         
 13  job                 100000 non

## **Descriptive**

In [230]:
df_eda.describe()

,id,cc_num,gender,zip,lat,long,city_pop,dob,acct_num,trans_date,trans_time,unix_time,amt,is_fraud,merch_lat,merch_long,distance
count,1.000000e+05,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05,100000,1.000000e+05,100000,100000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.733159e+07,3.687788e+17,0.482390,51192.157110,37.581594,-91.996846,2.933952e+05,1978-11-01 20:48:46.080000,4.971789e+11,2019-12-16 01:24:10.080000256,1900-01-01 14:02:32.631060224,1.576490e+09,67.459051,0.002890,37.584537,-91.997536,76.406411
min,1.700000e+01,6.040003e+10,0.000000,1002.000000,19.589300,-163.118900,4.400000e+01,1928-01-05 00:00:00,1.837412e+07,2019-12-01 00:00:00,1900-01-01 00:00:01,1.575139e+09,1.000000,0.000000,18.695408,-164.061908,0.319967
25%,8.656808e+06,3.852401e+13,0.000000,27565.000000,33.897200,-98.643300,1.785300e+04,1967-08-19 18:00:00,2.443508e+11,2019-12-08 00:00:00,1900-01-01 09:06:24,1.575797e+09,7.490000,0.000000,33.865926,-98.986065,55.484537
50%,1.732208e+07,3.506601e+15,0.000000,48150.000000,38.565600,-87.175700,6.076300e+04,1981-07-25 12:00:00,4.949888e+11,2019-12-16 00:00:00,1900-01-01 15:00:56.500000,1.576439e+09,34.500000,0.000000,38.453030,-87.053708,78.642078
75%,2.598662e+07,4.593295e+15,1.000000,78363.000000,41.224750,-79.939600,2.337170e+05,1992-11-15 00:00:00,7.501972e+11,2019-12-24 00:00:00,1900-01-01 19:31:22.249999872,1.577136e+09,79.280000,0.000000,41.306195,-79.671186,98.820498
max,3.463594e+07,4.999689e+18,1.000000,99929.000000,66.693300,-67.407300,2.906700e+06,2008-11-24 00:00:00,9.999583e+11,2019-12-31 00:00:00,1900-01-01 23:59:59,1.577817e+09,11078.860000,1.000000,66.482129,-66.921783,147.041384
std,1.000948e+07,1.232144e+18,0.499692,29728.394249,5.184296,16.506715,5.648002e+05,NaN,2.895940e+11,NaN,NaN,7.680531e+05,164.191618,0.053681,5.214922,16.515207,29.269574


In [219]:
df_eda['is_fraud'].value_counts()

is_fraud
0    99711
1      289
Name: count, dtype: int64

In [216]:
df_eda.columns

Index(['id', 'ssn', 'cc_num', 'first', 'last', 'gender', 'street', 'city',
       'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'acct_num',
       'profile', 'trans_num', 'trans_date', 'trans_time', 'unix_time',
       'category', 'amt', 'is_fraud', 'merchant', 'merch_lat', 'merch_long',
       'region', 'trans_time_segment', 'age_group', 'cc_type', 'area_cat'],
      dtype='object')

In [209]:
cols_of_interest = ['cc_num', 'gender', 'state', 'zip', 'city_pop', 'dob', 
       'trans_num', 'trans_date', 'trans_time', 'unix_time',
       'category', 'amt', 'is_fraud', 'region', 'trans_time_segment', 'age_group', 'cc_type', 'area_cat']

In [210]:
from autoviz.AutoViz_Class import AutoViz_Class

In [217]:
cols_of_interest = ['gender', 'city_pop', 'dob', 'trans_date', 'trans_time', 
                    'unix_time', 'category', 'amt', 'is_fraud', 'region', 
                    'trans_time_segment', 'age_group', 'cc_type', 'area_cat']

df_subset = df_eda[cols_of_interest]
df_subset.to_csv(fname + '_subset.csv', index=False)
df_subset.head()

,gender,city_pop,dob,trans_date,trans_time,unix_time,category,amt,is_fraud,region,trans_time_segment,age_group,cc_type,area_cat
0,1,139474,1970-10-23,2019-12-26,1900-01-01 06:36:03,1577322363,entertainment,3.65,0,Midwest,Morning_6-12,middle_aged,Maestro,urban
1,1,7123,1983-08-03,2019-12-28,1900-01-01 22:53:34,1577553814,shopping_pos,5.25,0,West,Evening_18-24,young_adult,Maestro,urban
2,1,92558,1980-04-27,2019-12-23,1900-01-01 13:23:24,1577087604,shopping_pos,25.31,0,West,Afternoon_12-18,young_adult,Unknown,urban
3,1,2297,1998-11-13,2019-12-03,1900-01-01 09:21:51,1575345111,shopping_pos,95.28,0,Midwest,Morning_6-12,young_adult,JCB,rural
4,1,20107,2001-07-19,2019-12-27,1900-01-01 21:56:44,1577464004,shopping_pos,73.33,0,Midwest,Evening_18-24,teenager,Maestro,urban


In [ ]:
## NOTE: Some faature engineering needs to be done before running AutoViz
# AutoViz for automated visualizations
AV = AutoViz_Class()
# Generate automated visualizations
dft = AV.AutoViz(
    filename = fname + '_subset.csv', 
    #dfte=df_subset,
    #depVar ='is_fraud',  # Target variable
    #chart_format = 'matplotlib',
    #max_rows_analyzed=10000,  # Limit for performance
    #max_cols_analyzed=15    
)

## ***add some features*** 
see fraud distribution by age_group, time_group, gender, category, distance, job, etc.


In [224]:
def plot_categorical(df_in, col, target='is_fraud', top_n=10):
    """
    Plot a bar chart of the counts of the top_n categories in a column.
    """
    # Get the top_n categories
    top_n_cats = df_in[col].value_counts().head(top_n).index
    # Create a bar plot
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df_in, y=col, order=top_n_cats, hue=target)
    plt.title(f'Count of {col} with Fraud')
    plt.show()

def plot_numerical(df_in, col, target='is_fraud', bins=30):
    """
    Plot a histogram of a numerical column.
    """
    # Create a histogram    
    plt.figure(figsize=(10, 6))
    sns.histplot(df_in, x=col, bins=bins, hue=target, kde=True)
    plt.title(f'{col} Distribution with Fraud')
    plt.show()

def plot_categorical_percent(df_in, col, target='is_fraud', top_n=10):
    """
    Plot a bar chart of the percentage of the top_n categories in a column.
    """
    # Get the top_n categories  
    top_n_cats = df_in[col].value_counts().head(top_n).index
    # Create a bar plot
    plt.figure(figsize=(10, 6))
    (df_in[col].value_counts() / len(df_in)).head(top_n).plot(kind='bar')
    plt.title(f'Percentage of {col}')
    plt.show()

def plot_numerical_percent(df_in, col, target='is_fraud', bins=30):
    """
    Plot a histogram of a numerical column.
    """
    # Create a histogram
    plt.figure(figsize=(10, 6))
    df_in[col].plot(kind='hist', bins=bins)

    plt.title(f'{col} Distribution')
    plt.show()



Generate distance between the CC user and the merchant

In [ ]:
# Function to calculate distance between user and merchant locations
def calculate_distance(row):
    user_location = (row['lat'], row['long'])
    merchant_location = (row['merch_lat'], row['merch_long'])
    return geodesic(user_location, merchant_location).kilometers

# Apply distance calculation
df_eda['distance'] = df_eda.apply(calculate_distance, axis=1)


In [ ]:
#print(df_eda['distance'].value_counts())
print(df_eda['distance'].describe())
print('skewness', df_eda['distance'].skew()) # -0.24, largely symmetrical distribution with a slight pull to the left

# Display the distribution of distances 
plt.figure(figsize=(10, 6))
sns.histplot(df_eda['distance'], bins=30, kde=True)

distance_bins = [0, 20, 50, 100, 150]  # Ranges for categorization
distance_labels = ["local_0-20_km", "close_21-50_km", "moderate_51-100_km", "far_101-150_km"]

# Create a new column with distance categories
df_eda['distance_category'] = pd.cut(df_eda['distance'], bins=distance_bins, labels=distance_labels, include_lowest=True)

#df_eda.head()

Time since last transaction, based on unix_time

In [ ]:
#Time since last transaction, based on unix_time
df_eda = df_eda.sort_values(['acct_num', 'unix_time'])
df_eda['time_since_last_trans'] = df_eda.groupby('acct_num')['unix_time'].diff() / 3600  # in hours
df_eda['time_since_last_trans'] = df_eda['time_since_last_trans'].fillna(0)

 Narrow Job categories  

In [229]:
# Define the mapping function
def map_job_to_category_by_keyword(job):
    
    # Handle non-string or missing inputs
    job = job.lower().strip() if isinstance(job, str) else ''
    
    # Define categories by their associated keywords
    category_keywords = {
        "Finance": ["finance", "bank", "investment", "accountant", "cfo","financial","economist","loan"],   
        "Engineering": ["engineer", "mechanical", "electrical", "manufacturing", "site"],
        "Education": ["teacher", "educator", "librarian", "professor", "instructor", "tutor","lecturer","tutor","trainer",
                      "instructional","education","principal","school","curriculum"],        
        "Design": ["designer", "graphic", "fashion", "exhibition", "interior"],
        "Healthcare": ["nurse", "doctor", "physician", "medical", "health", 'immunologist', 'surgeon', "clinical", "pharmacist", "optometrist","dentist", 
                       "physiotherapist", "oncologist", "radiologist", "neurologist", "cardiologist", "pediatrician", "endocrinologist", "urologist", 
                       "anesthesiologist", "dietitian", "orthodontist"],
        "Legal": ["lawyer", "attorney", "legal", "counsel", "paralegal", "solicitor", "barrister", "advocate", "notary","judge"],
        "Science": ["scientist", "biomedical", "marine", "forensic", "laboratory"],
        "Surveying": ["surveyor", "quantity", "land", "planning", "development"],
        "Therapy": ["therapist", "speech", "art", "occupational", "psychologist","physiotherapist", "counselor", "chiropractor", "psychiatrist"],
        "Hospitality": ["hotel", "restaurant", "catering", "event", "tourism","chef","bartender","event","food","beverage","sommelier","pastry"],
        "Management": ["manager", "director", "executive", "supervisor", "chief","executive","director","management"],
        "Media": ["journalist", "editor", "reporter", "broadcast", "media","printmaker","graphic","photographer","videographer",
                  "illustrator","animator","film","content","editor","copywriter",],
        "Architecture": ["architect", "structural", "landscape", "building"],
        "Administration": ["administrator", "officer", "coordinator", "clerk", "assistant"],
        "Consulting": ["consultant", "advisor", "analyst", "strategist", "specialist"],
        "Technical_IT": ["technician", "it", "technical", "support", "maintenance", "software","data","cybersecurity","system","network",
                         "telecom","ai","devops","database","web","cloud","game"],
        "Other": []  # Default category if no keywords match
    }
    
    # Check each category for a keyword match
    for category, keywords in category_keywords.items():
        if any(keyword in job for keyword in keywords):
            return category
    
    # If no keywords match, assign to Miscellaneous
    return "Other"


# Handle missing values in the 'job' column
df_eda['job'] = df_eda['job'].fillna('Unknown')

# Apply the mapping function to the 'job' column to create 'job_cat'
df_eda['job_cat'] = df_eda['job'].apply(map_job_to_category_by_keyword)

#test
#df.to_csv('job_cat.csv', index=False)

print("\nJob Category Distribution:")
print(df_eda['job_cat'].value_counts())


Job Category Distribution:
job_cat
Other             18685
Engineering       10071
Healthcare         9034
Management         8505
Administration     7420
Therapy            6703
Education          6388
Technical_IT       5399
Surveying          5061
Media              4972
Design             4646
Science            3894
Finance            3427
Legal              2282
Consulting         1478
Hospitality        1321
Architecture        714
Name: count, dtype: int64


In [232]:
df_eda['trans_day_of_week'] = df_eda['trans_date'].dt.dayofweek  # 0 = Monday, 6 = Sunday
#df_eda['trans_hour'] = pd.to_datetime(df_eda['trans_time'].astype(str)).dt.hour
df_eda['is_weekend'] = df_eda['trans_day_of_week'].isin([5, 6]).astype(int)  # 1 = weekend, 0 = weekday

In [235]:
# Amount categories
bins = [0, 10, 50, 100, 500, float('inf')]
labels = ['very_low', 'low', 'medium', 'high', 'very_high']
df_eda['amt_category'] = pd.cut(df['amt'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
#is_fraud by amount category
plt.figure(figsize=(10, 6))
sns.countplot(data=df_eda, x='amt_category', hue='is_fraud')
plt.title('Count of Fraud by Amount Category')
plt.show()

In [238]:
df_eda.columns


Index(['id', 'ssn', 'cc_num', 'first', 'last', 'gender', 'street', 'city',
       'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'acct_num',
       'profile', 'trans_num', 'trans_date', 'trans_time', 'unix_time',
       'category', 'amt', 'is_fraud', 'merchant', 'merch_lat', 'merch_long',
       'region', 'trans_time_segment', 'age_group', 'cc_type', 'area_cat',
       'distance', 'distance_category', 'job_cat', 'trans_day_of_week',
       'trans_hour', 'is_weekend', 'amt_category'],
      dtype='object')

In [239]:
df_eda.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long,region,trans_time_segment,age_group,cc_type,area_cat,distance,distance_category,job_cat,trans_day_of_week,trans_hour,is_weekend,amt_category
0,18832098,087-12-2118,639017474813,Zachary,Parks,1,94238 Jacob Cliff Apt. 458,Columbia,MO,65203,38.9348,-92.3639,139474,designer exhibition/display,1970-10-23,941725909341,adults_50up_male_urban.json,a36eecc709911793fe140ba7d29404ca,2019-12-26,1900-01-01 06:36:03,1577322363,entertainment,3.65,0,"turcotte, mckenzie and koss",37.978633,-91.813002,Midwest,Morning_6-12,middle_aged,Maestro,urban,116.522952,far_101-150_km,Design,3,6,0,very_low
1,24801519,267-50-6963,6567701540795312,Jeremy,Soto,1,29865 Bush Centers,Goldendale,WA,98620,45.8320,-120.8130,7123,psychiatric nurse,1983-08-03,856035171395,adults_2550_male_urban.json,5c0acce002c0e0b56de66d9de0cbff80,2019-12-28,1900-01-01 22:53:34,1577553814,shopping_pos,5.25,0,hermann and sons,45.829641,-121.389115,West,Evening_18-24,young_adult,Maestro,urban,44.764257,close_21-50_km,Healthcare,5,22,1,very_low
2,31141574,411-62-3428,180022197664580,Samuel,Mcdonald,1,4570 Ruth Harbors,Palo Alto,CA,94306,37.4180,-122.1274,92558,occupational psychologist,1980-04-27,401937847226,adults_2550_male_urban.json,c363df118159e47c9a46ff906471c0c0,2019-12-23,1900-01-01 13:23:24,1577087604,shopping_pos,25.31,0,conroy ltd,37.825064,-121.644324,West,Afternoon_12-18,young_adult,Unknown,urban,62.128431,moderate_51-100_km,Therapy,0,13,0,low
3,25784982,841-48-6189,3584095374557286,Nicholas,Davis,1,83759 Shannon Meadows,Kohler,WI,53044,43.7381,-87.7867,2297,hospital pharmacist,1998-11-13,497586259866,adults_2550_male_rural.json,e33a7e2168a80e7b3a9c7ba239219259,2019-12-03,1900-01-01 09:21:51,1575345111,shopping_pos,95.28,0,hudson-grady,43.686522,-86.947295,Midwest,Morning_6-12,young_adult,JCB,rural,67.891933,moderate_51-100_km,Healthcare,1,9,0,medium
4,19821729,603-28-2907,6011226654586127,Mark,Sanchez,1,21975 Pam Garden,Birmingham,MI,48009,42.5444,-83.2133,20107,medical physicist,2001-07-19,765945154350,young_adults_male_urban.json,bf6116234aaf567b13a629cf6b604b0b,2019-12-27,1900-01-01 21:56:44,1577464004,shopping_pos,73.33,0,dooley inc,43.031478,-83.903585,Midwest,Evening_18-24,teenager,Maestro,urban,78.215297,moderate_51-100_km,Healthcare,4,21,0,medium


In [ ]:
# drop columns to ease working with the data
cat_cols = ['gender', 'city_pop', 'job', 'dob', 'acct_num',
       'profile', 'trans_num', 'trans_date', 'trans_time', 'unix_time',
       'category', 'amt', 'is_fraud', 'merchant', 'merch_lat', 'merch_long',
       'region', 'trans_time_segment', 'age_group', 'cc_type', 'area_cat',
       'distance', 'distance_category', 'job_cat', 'trans_day_of_week',
       'trans_hour', 'is_weekend', 'amt_category'],

##### Categorical